## Access Our Data

### 1. Download it from OneDrive using this [link](https://liveuclac-my.sharepoint.com/:f:/g/personal/ucabdsm_ucl_ac_uk/Ek8oM7sBaJhFs5Py5FPdtYEB_YW0TcEVvtjjdtNqp_Ud4A?e=Alw6kD). 
### 2. Move the 'comp0241_data.zip' file into the same directory as the current notebook and unzip it.
### 3. Make sure you have the 'comp0241_data' folder in this directory.
### 4. Run the code below.

# Task 1
##  Extract the Astronomical Object from Images

### a) Implement at Least Two Methods for AO Extraction
### b) Combine Methods for Improved Accuracy
### c) Evaluate Performance with ROC Curves

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import os

def align_images_multiple_references(reference_image_paths, distorted_image_path):
    # Load the distorted image
    dist_img = cv2.imread(distorted_image_path)
    dist_gray = cv2.cvtColor(dist_img, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT
    sift = cv2.SIFT_create()

    # Aggregate keypoints and descriptors from all reference images
    ref_keypoints = []
    ref_descriptors = []
    for ref_path in reference_image_paths:
        ref_img = cv2.imread(ref_path)
        ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = sift.detectAndCompute(ref_gray, None)
        ref_keypoints.extend(keypoints)
        if descriptors is not None:
            if len(ref_descriptors) == 0:
                ref_descriptors = descriptors
            else:
                ref_descriptors = np.vstack((ref_descriptors, descriptors))

    # Detect keypoints and descriptors in the distorted image
    keypoints_dist, descriptors_dist = sift.detectAndCompute(dist_gray, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(ref_descriptors, descriptors_dist)

    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    ref_pts = np.float32([ref_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dist_pts = np.float32([keypoints_dist[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate homography
    matrix, mask = cv2.findHomography(dist_pts, ref_pts, cv2.RANSAC, 5.0)

    # Warp the distorted image
    h, w = dist_img.shape[:2]
    aligned_image = cv2.warpPerspective(dist_img, matrix, (w, h))

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Distorted Image")
    plt.imshow(cv2.cvtColor(dist_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Aligned Image")
    plt.imshow(cv2.cvtColor(aligned_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    return matrix

def transform_image_with_homography(image_path, homography_matrix):
    # Load the image
    image = cv2.imread(image_path)
    h, w = image.shape[:2]  # Get the size from the image itself

    # Apply the homography transformation
    transformed_image = cv2.warpPerspective(image, homography_matrix, (w, h))

    return transformed_image

def reverse_transform_image_with_homography(transformed_image, homography_matrix):
    """
    Reverse the homography transformation on an image.

    Parameters:
        transformed_image: The image that was transformed.
        homography_matrix: The homography matrix used for the forward transformation.

    Returns:
        The reversed (original) image.
    """
    # Calculate the inverse of the homography matrix
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    
    # Get the size of the transformed image
    h, w = transformed_image.shape[:2]
    
    # Apply the inverse homography transformation
    original_image = cv2.warpPerspective(transformed_image, inverse_homography_matrix, (w, h))
    
    return original_image

def save_homography_matrix(matrix, file_path):
    # Convert the NumPy array to a list for JSON serialization
    matrix_list = matrix.tolist()
    with open(file_path, 'w') as f:
        json.dump(matrix_list, f)

def process_dataset_images(dataset_folder):
    images_folder = os.path.join(dataset_folder, "images")
    
    for image_name in os.listdir(images_folder):
        print(image_name)
        image_path = os.path.join(images_folder, image_name)

        reference_image_paths = [
            './Dataset 2/images/000003.png', './Dataset 2/images/000010.png', './Dataset 2/images/000016.png', './Dataset 2/images/000021.png', './Dataset 2/images/000022.png', './Dataset 2/images/000028.png', './Dataset 2/images/000030.png', './Dataset 2/images/000033.png', './Dataset 2/images/000046.png', './Dataset 2/images/000051.png', './Dataset 2/images/000053.png', './Dataset 2/images/000060.png', './Dataset 2/images/000061.png', './Dataset 2/images/000064.png',
            './Dataset 2/images/000072.png', './Dataset 2/images/000081.png', './Dataset 2/images/000096.png', 
        ]  # Add more references
        #distorted_image_path = './Dataset 2/images/000083.png'
        homography_matrix = align_images_multiple_references(reference_image_paths, image_path)

        # Transform the image
        transformed_image = transform_image_with_homography(image_path, homography_matrix)

        if not cv2.imwrite(f"./calibrated-images/{image_name}", transformed_image):
            print(f"Failed to save image: {image_name}")

        save_homography_matrix(homography_matrix, f"./calibrated-images/{image_name.replace('png', 'json')}")

        # Plot the original and transformed images
        #original_image = cv2.imread(image_path)
        #plt.figure(figsize=(12, 6))
        #plt.subplot(1, 2, 1)
        #plt.title("Original Image")
        #plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        #plt.axis("off")

        #plt.subplot(1, 2, 2)
        #plt.title("Transformed Image")
        #plt.imshow(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        #plt.axis("off")

        #plt.tight_layout()
        #plt.show()

# Example usage
dataset_folder = './Dataset 2'  # Replace with the actual dataset folder
process_dataset_images(dataset_folder)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import os
import json

def load_homography_matrix(file_path):
    # Load the JSON file and convert the list back to a NumPy array
    with open(file_path, 'r') as f:
        matrix_list = json.load(f)
    return np.array(matrix_list)

def reverse_transform_image_with_homography(transformed_image, homography_matrix):
    """
    Reverse the homography transformation on an image.

    Parameters:
        transformed_image: The image that was transformed.
        homography_matrix: The homography matrix used for the forward transformation.

    Returns:
        The reversed (original) image.
    """
    # Calculate the inverse of the homography matrix
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    
    # Get the size of the transformed image
    h, w = transformed_image.shape[:2]
    
    # Apply the inverse homography transformation
    original_image = cv2.warpPerspective(transformed_image, inverse_homography_matrix, (w, h))
    
    return original_image

def color_thresholding(image, lower_bounds, upper_bounds):
    """
    Perform color thresholding to segment the globe.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.
        lower_bounds (list of numpy.ndarray): List of lower HSV color bounds.
        upper_bounds (list of numpy.ndarray): List of upper HSV color bounds.

    Returns:
        numpy.ndarray: Binary mask highlighting the selected colors.
    """
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    # Apply each color threshold and combine the masks
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

def refined_circle_detection_with_mask(image, color_mask):
    """
    Detect a single circular shape in the image using a color mask to focus the detection.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.
        color_mask (numpy.ndarray): Binary mask obtained from color thresholding.

    Returns:
        numpy.ndarray: Binary mask with the detected circle.
    """
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    # Convert to grayscale
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)
    plt.figure(figsize=(6, 6))
    plt.imshow(blurred_image, cmap='gray')
    plt.title(f"Thresholding Result {i:06d}")
    plt.axis("off")
    plt.show()
    # Detect circles using HoughCircles with adjusted parameters
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=0.2,
        minDist=200,
        param1=150,
        param2=40,
        minRadius=30,
        maxRadius=400,
    )

    # Create a binary mask with the same dimensions as the input image
    mask = np.zeros_like(gray_image)
    if circles is not None:
        # Get the first (most prominent) circle
        circle = np.round(circles[0, 0]).astype("int")
        x, y, r = circle
        # Draw the circle on the mask
        cv2.circle(mask, (x, y), r, 255, -1)
    return mask

def compute_roc_curve(predicted_mask, ground_truth_mask):
    """
    Compute ROC curve and AUC for a predicted mask and ground truth mask.

    Parameters:
        predicted_mask (numpy.ndarray): Predicted binary mask.
        ground_truth_mask (numpy.ndarray): Ground truth binary mask.

    Returns:
        tuple: (fpr, tpr, roc_auc) False Positive Rate, True Positive Rate, and Area Under Curve.
    """
    # Binarize the masks
    predicted_binary = (predicted_mask > 0).astype(np.uint8).flatten()
    ground_truth_binary = (ground_truth_mask > 0).astype(np.uint8).flatten()

    # Compute ROC curve
    fpr, tpr, _ = roc_curve(ground_truth_binary, predicted_binary)
    roc_auc = auc(fpr, tpr)

    return fpr, tpr, roc_auc

# Dataset paths
images_folder = './Dataset 2/images/'
corrected_images_folder = './calibrated-images/'
masks_folder = './Dataset 2/masks/'

# Bounds for color thresholding
lower_bounds = [
    np.array([100, 50, 50]),  # Lower bound of blue in HSV
    np.array([0, 0, 200]),    # Lower bound of white in HSV
]
upper_bounds = [
    np.array([140, 255, 255]),  # Upper bound of blue in HSV
    np.array([180, 50, 255]),   # Upper bound of white in HSV
]

# Lists to store ROC values
all_fpr = []
all_tpr = []
all_auc = []

# Loop through all images and masks
for i in range(100):
    image_path = os.path.join(images_folder, f"{i:06d}.png")
    corrected_image_path = os.path.join(corrected_images_folder, f"{i:06d}.png")
    mask_path = os.path.join(masks_folder, f"{i:06d}.png")

    # Read image and ground truth mask
    image = cv2.imread(image_path)
    corrected_image = cv2.imread(corrected_image_path)
    ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    loaded_matrix = load_homography_matrix(corrected_image_path.replace("png", "json"))

    # Generate color mask
    color_mask = color_thresholding(corrected_image, lower_bounds, upper_bounds)

    # Plot the thresholding result
    plt.figure(figsize=(6, 6))
    plt.imshow(color_mask, cmap='gray')
    plt.title(f"Thresholding Result {i:06d}")
    plt.axis("off")
    plt.show()

    # Refine circle detection with color mask
    predicted_mask = reverse_transform_image_with_homography(refined_circle_detection_with_mask(image, color_mask), loaded_matrix)

    if not cv2.imwrite(f"./calibrated-images/{i:06d}_predicted_mask.png", predicted_mask):
            print(f"Failed to save {i:06d}_predicted_mask.png")

    # Compute ROC curve
    fpr, tpr, roc_auc = compute_roc_curve(predicted_mask, ground_truth_mask)

    # Store the results
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_auc.append(roc_auc)

    # Plot image, ground truth mask, and predicted mask
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Image {i:06d}")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(ground_truth_mask, cmap='gray')
    plt.title("Ground Truth Mask")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Compute average ROC curve (mean TPR for each FPR)
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.zeros_like(mean_fpr)
for fpr, tpr in zip(all_fpr, all_tpr):
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
mean_tpr /= len(all_tpr)
mean_auc = auc(mean_fpr, mean_tpr)

# Plot average ROC curve
plt.figure()
plt.plot(mean_fpr, mean_tpr, label=f"Mean ROC (AUC = {mean_auc:.2f})")
plt.title("Average ROC Curve for Test Set")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

# Print mean AUC
print(f"Mean AUC: {mean_auc:.2f}")


# Task 2
## Measure the Projection Point of the Rotation Axis and the Height of the AO

### a) Determine the Geometric Centre in Images

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Dataset paths
images_folder = './Dataset 2/images/'
masks_folder = './calibrated-images/'

def find_mask_center(mask):
    # Compute the coordinates of the white pixels
    white_pixels = np.column_stack(np.where(mask == 255))

    if len(white_pixels) == 0:
        print(f"No white pixels found in mask for index {i}, skipping.")
        return None, None

    # Compute the mean (center location) of the white pixels
    center_y, center_x = white_pixels.mean(axis=0)
    return center_x, center_y

for i in range(100):
    image_path = os.path.join(images_folder, f"{i:06d}.png")
    mask_path = os.path.join(masks_folder, f"{i:06d}_predicted_mask.png")

    # Read image and predicted mask
    image = cv2.imread(image_path)
    predicted_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None or predicted_mask is None:
        print(f"Missing image or mask for index {i}, skipping.")
        continue

    center_x, center_y = find_mask_center(predicted_mask)

    # Plot image and predicted mask
    plt.figure(figsize=(8, 4))

    # Plot the original image with the center marked
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.scatter([center_x], [center_y], color='red', s=40, label='Center')
    plt.title("Original Image with Center")
    plt.legend()
    plt.axis('off')

    # Plot the predicted mask
    plt.subplot(1, 2, 2)
    plt.imshow(predicted_mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


### b) Assess the Movement of the Centre Over Time 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path to the video
video_path = './single_rotation_top_view.mp4'

# Open the video file
video_capture = cv2.VideoCapture(video_path)

# Check if video was successfully opened
if not video_capture.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
max_images = 100  # Maximum number of frames to process
interval = max(1, total_frames // max_images)  # Interval to skip frames

print(f"Total frames: {total_frames}, Processing every {interval}th frame.")

# Initialize variables
frame_count = 0
processed_count = 0
all_centers = []
first_frame = None

# Lower and upper bounds for color thresholding
lower_bounds = [
    np.array([10, 0, 0]),  # Example for specific hue
    np.array([0, 0, 200])  # Bright whites
]
upper_bounds = [
    np.array([150, 250, 200]),  # Saturated color range
    np.array([180, 50, 255])    # Bright whites
]

# Helper functions
def color_thresholding(image, lower_bounds, upper_bounds):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

def refined_circle_detection_with_mask(image, color_mask):
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=0.2,
        minDist=200,
        param1=150,
        param2=40,
        minRadius=30,
        maxRadius=400,
    )
    mask = np.zeros_like(gray_image)
    if circles is not None:
        circle = np.round(circles[0, 0]).astype("int")
        x, y, r = circle
        cv2.circle(mask, (x, y), r, 255, -1)
    return mask

def find_mask_center(binary_mask):
    moments = cv2.moments(binary_mask)
    if moments["m00"] != 0:
        center_x = int(moments["m10"] / moments["m00"])
        center_y = int(moments["m01"] / moments["m00"])
        return center_x, center_y
    return None

# Process the video frame by frame
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break  # Break if no more frames

    if frame_count % interval == 0:
        # Store the first frame for visualization
        if processed_count == 0:
            first_frame = frame.copy()

        # Apply color thresholding
        binary_mask = color_thresholding(frame, lower_bounds, upper_bounds)

        # Apply refined circle detection
        predicted_mask = refined_circle_detection_with_mask(frame, binary_mask)

        # Find the center of the binary mask
        center = find_mask_center(predicted_mask)
        if center:
            all_centers.append(center)

        processed_count += 1

        # Stop after processing the maximum number of frames
        if processed_count >= max_images:
            break

    frame_count += 1

# Release the video capture
video_capture.release()

# Process results if a valid first frame exists
if first_frame is not None:
    # Convert centers to integer coordinates
    int_centers = [tuple(map(int, center)) for center in all_centers]

    # Find the most left and most right centers
    most_left = min(int_centers, key=lambda point: point[0])  # Minimum x-coordinate
    most_right = max(int_centers, key=lambda point: point[0])  # Maximum x-coordinate

    # Draw a red line between the most left and most right points
    cv2.line(first_frame, most_left, most_right, (0, 0, 255), 2)  # Red line

    # Show the result
    plt.figure(figsize=(10, 6))
    plt.title("Swing Motion Indicator (Left to Right)")
    plt.imshow(cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

print(f"Processed {processed_count} frames.")

### c) Estimate the AO's Height Above Ground

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Functions for calculations
def find_camera_params(size_pixels, size_meters, distance_meters, image_width, image_height):
    # Calculate the focal length in pixels
    focal_length = (size_pixels * distance_meters) / size_meters
    print(f"Focal length in pixels: {focal_length:.2f}")

    # Assume the camera principal point (cx, cy) is the image center
    cx = image_width / 2
    cy = image_height / 2

    # Intrinsic camera matrix
    camera_matrix = np.array([
        [focal_length, 0, cx],
        [0, focal_length, cy],
        [0, 0, 1]
    ])

    return camera_matrix, focal_length

def find_real_size(focal_length, size_pixel, distance_meters):
    return (size_pixel * distance_meters) / focal_length

def find_distance(focal_length, size_pixel, size_real):
    return (focal_length * size_real) / size_pixel

# Function to perform color thresholding
def color_thresholding(image, lower_bounds, upper_bounds):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

# Function to perform refined circle detection
def refined_circle_detection_with_mask(image, color_mask):
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (11, 11), 2)
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=0.2,
        minDist=200,
        param1=150,
        param2=40,
        minRadius=30,
        maxRadius=400,
    )
    if circles is not None:
        return np.uint16(np.around(circles))[0, 0]  # First circle (x, y, radius)
    return None

# Inputs
image_path = "2c.jpg"
resize_factor = 0.25  # Resize image to speed up processing

focal_length = 5492.23
diameter_meters = 7.0
distance_of_the_camera_from_floor = 1.80

# Lower and upper bounds for color thresholding
lower_bounds = [
    np.array([10, 0, 0]),  # Example for specific hue
    np.array([0, 0, 200])  # Bright whites
]
upper_bounds = [
    np.array([150, 250, 200]),  # Saturated color range
    np.array([180, 50, 255])    # Bright whites
]

# Read and resize the image
frame = cv2.imread(image_path)
if frame is None:
    raise ValueError("Unable to read the image.")
height, width, channels = frame.shape
frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

# Apply color thresholding
binary_mask = color_thresholding(frame, lower_bounds, upper_bounds)

# Apply refined circle detection
circle_params = refined_circle_detection_with_mask(frame, binary_mask)
if circle_params is None:
    raise ValueError("No circle detected.")
x, y, r = circle_params  # Center (x, y) and radius

# Draw the circle and center
output_image = frame.copy()
cv2.circle(output_image, (x, y), r, (0, 255, 0), 2)  # Globe circle
cv2.circle(output_image, (x, y), 5, (0, 0, 255), -1)  # Center point

# Plot horizontal diameter
left_x = x - r
right_x = x + r
cv2.line(output_image, (left_x, y), (right_x, y), (255, 0, 0), 2)  # Diameter line

# Calculate and display diameter
diameter_pixels = 2 * r
print(f"Diameter in pixels (after resizing): {diameter_pixels}")

# Scale back to original size for correct interpretation
original_diameter_pixels = diameter_pixels / resize_factor
print(f"Diameter in pixels (original resolution): {original_diameter_pixels}")

distance_to_globe = find_distance(focal_length, original_diameter_pixels, diameter_meters) + distance_of_the_camera_from_floor
print(f"Distance to globe: {distance_to_globe:.4f} meters")

# Visualize
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title(f"Globe Detection\nDistance to globe: {distance_to_globe:.2f} m")
plt.axis("off")
plt.show()


# Task 3
## Estimate the Rotation Cycle of the AO

### a) Explain Your Methodology
### b) Provide a Single Rotation Cycle Estimate as reference

In [ ]:
# No code needed here, the explanation is provided in the report.

### c) Continuous Rotation Cycle Estimation from Video
### d) Real-Time Rotation Cycle Estimation
### e) Compare Estimations from Different Views

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
from scipy.signal import find_peaks

video_paths = ["single_rotation_top_view.mp4", "single_rotation_bottom_view.mp4", "multiple_rotations_top_view.mp4"]

for video_path in video_paths:
    print(f"Processing video: {video_path}")
    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frame rate: {frame_rate} fps")
    frame_interval = 10  # Process every 10th frame

    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        raise ValueError("Unable to read video.")

    # Define ROI (manually set or use a selection tool beforehand)
    x, y, w, h = [599, 180, 489, 449]  # Example ROI
    if w == 0 or h == 0:
        raise ValueError("No valid ROI selected.")

    # Extract ROI and detect features in the first frame
    first_roi = first_frame[y:y+h, x:x+w]
    gray_first_roi = cv2.cvtColor(first_roi, cv2.COLOR_BGR2GRAY)

    # Detect features in the first frame using SIFT
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray_first_roi, None)

    frame_idx = 0

    # Store good matches count for each processed frame
    matches_counts = []
    processed_frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video stream

        frame_idx += 1

        # Skip frames to process every 10th frame
        if frame_idx % frame_interval != 0:
            continue

        start = time.perf_counter()

        # Extract ROI from the current frame
        roi_frame = frame[y:y+h, x:x+w]
        gray_roi_frame = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

        # Detect features in the current frame
        kp2, des2 = sift.detectAndCompute(gray_roi_frame, None)
        if des2 is None:
            matches_counts.append(0)
            processed_frames.append(frame_idx)
            continue

        # Match features using FLANN
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1, des2, k=2)

        # Apply Lowe's ratio test
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

        # Record the number of good matches
        matches_counts.append(len(good_matches))
        processed_frames.append(frame_idx)

        end = time.perf_counter()
        elapsed = end - start

        print(f"Frame {frame_idx}: {len(good_matches)} good matches, elapsed: {elapsed:.3f}s")

    cap.release()

    # Convert frame indices to time
    processed_times = np.array(processed_frames) / frame_rate

    # Find peaks in the matches_counts array
    # Adjust parameters as needed (e.g., prominence, height) to reliably find peaks
    peaks, _ = find_peaks(matches_counts, prominence=100, distance=50)  # Try different prominence values
    peaks = np.insert(peaks, 0, 0.0)

    if len(peaks) < 2:
        print("Not enough peaks found to estimate multiple rotations.")
    else:
        
        # Calculate rotation periods between consecutive peaks
        rotation_times = processed_times[peaks]
        intervals = np.diff(rotation_times)  # time differences between peaks

        # Number of rotations is basically the number of peaks - 1 (assuming the first peak as the first complete rotation)
        num_rotations = len(peaks) - 1
        avg_rotation_period = np.mean(intervals) if num_rotations > 0 else None

        print(f"Detected {len(peaks)} peaks at times: {rotation_times}")
        if avg_rotation_period is not None:
            print(f"Number of full rotations: {num_rotations}")
            print(f"Average rotation period: {avg_rotation_period:.2f} seconds")
        else:
            print("Only one peak found, cannot compute rotation period.")

    # Plotting the matches over time
    plt.figure(figsize=(10, 5))
    plt.plot(processed_times, matches_counts, label=f'Number of matches')
    if len(peaks) > 0:
        plt.plot(processed_times[peaks], np.array(matches_counts)[peaks], 'ro', label='Peaks')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Number of Matches')
    plt.title(f'[{video_path}] Number of full rotations: {num_rotations}, Average rotation period: {avg_rotation_period:.2f} seconds')
    plt.legend()
    plt.grid(True)
    plt.show()

# Task 4
# Bonus Objective: Estimate the Landing Speed in the Earth's Coordinate Frame of the Drone

### a) Estimate the AO's Diameter

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Functions for calculations
def find_camera_params(diameter_pixels, diameter_meters, distance_meters, image_width, image_height):
    # Calculate the focal length in pixels
    focal_length = (diameter_pixels * distance_meters) / diameter_meters
    print(f"Focal length in pixels: {focal_length:.2f}")

    # Assume the camera principal point (cx, cy) is the image center
    cx = image_width / 2
    cy = image_height / 2

    # Intrinsic camera matrix
    camera_matrix = np.array([
        [focal_length, 0, cx],
        [0, focal_length, cy],
        [0, 0, 1]
    ])

    return camera_matrix, focal_length

def find_real_size(focal_length, size_pixel, distance_meters):
    return (size_pixel * distance_meters) / focal_length

def find_distance(focal_length, size_pixel, size_real):
    return (focal_length * size_real) / size_pixel

# Function to perform color thresholding
def color_thresholding(image, lower_bounds, upper_bounds):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

# Function to perform refined circle detection
def refined_circle_detection_with_mask(image, color_mask):
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (11, 11), 2)
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=50,
        maxRadius=500,
    )
    if circles is not None:
        return np.uint16(np.around(circles))[0, 0]  # First circle (x, y, radius)
    return None

# Inputs
image_path = "4ab.jpg"
resize_factor = 0.25  # Resize image to speed up processing

# Camera matrix and focal length
#(array([[1.93142857e+03, 0.00000000e+00, 2.16000000e+03],
#        [0.00000000e+00, 1.93142857e+03, 3.84000000e+03],
#        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
# 1931.4285714285713)
distance_to_globe = 5.0  # Distance to the globe in meters
focal_length = 1931.43

# Lower and upper bounds for color thresholding
lower_bounds = [
    np.array([10, 0, 0]),  # Example for specific hue
    np.array([0, 0, 200])  # Bright whites
]
upper_bounds = [
    np.array([150, 250, 200]),  # Saturated color range
    np.array([180, 50, 255])    # Bright whites
]

# Read and resize the image
frame = cv2.imread(image_path)
if frame is None:
    raise ValueError("Unable to read the image.")
height, width, channels = frame.shape
frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

# Apply color thresholding
binary_mask = color_thresholding(frame, lower_bounds, upper_bounds)

# Apply refined circle detection
circle_params = refined_circle_detection_with_mask(frame, binary_mask)
if circle_params is None:
    raise ValueError("No circle detected.")
x, y, r = circle_params  # Center (x, y) and radius

# Draw the circle and center
output_image = frame.copy()
cv2.circle(output_image, (x, y), r, (0, 255, 0), 2)  # Globe circle
cv2.circle(output_image, (x, y), 5, (0, 0, 255), -1)  # Center point

# Plot horizontal diameter
left_x = x - r
right_x = x + r
cv2.line(output_image, (left_x, y), (right_x, y), (255, 0, 0), 2)  # Diameter line

# Calculate and display diameter
diameter_pixels = 2 * r
print(f"Diameter in pixels (after resizing): {diameter_pixels}")

# Scale back to original size for correct interpretation
original_diameter_pixels = diameter_pixels / resize_factor
print(f"Diameter in pixels (original resolution): {original_diameter_pixels}")

# Calculate real size (validation)
real_diameter_calculated = find_real_size(focal_length, original_diameter_pixels, distance_to_globe)
print(f"Real diameter calculated: {real_diameter_calculated:.4f} meters")

# Visualize
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title(f"Globe Detection\nDiameter: {real_diameter_calculated:.2f} m")
plt.axis("off")
plt.show()


### b) Evaluate Radius Consistency

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Functions for calculations
def find_real_size(focal_length, size_pixel, distance_meters):
    return (size_pixel * distance_meters) / focal_length

# Function to perform color thresholding
def color_thresholding(image, lower_bounds, upper_bounds):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

# Refined circle detection to create a mask
def refined_circle_detection_with_mask(image, color_mask):
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=0.2,
        minDist=200,
        param1=150,
        param2=40,
        minRadius=30,
        maxRadius=400,
    )
    mask = np.zeros_like(gray_image)
    if circles is not None:
        circle = np.round(circles[0, 0]).astype("int")
        x, y, r = circle
        cv2.circle(mask, (x, y), r, 255, -1)
    return mask

# Find center and radii based on the mask
def find_mask_center_and_radii(mask):
    white_pixels = np.column_stack(np.where(mask == 255))
    if len(white_pixels) == 0:
        return None, None, None, None, None, None

    # Find center
    center_y, center_x = white_pixels.mean(axis=0)
    center_x, center_y = int(center_x), int(center_y)

    # Compute radii
    # Left and right
    row_pixels = np.where(mask[center_y, :] == 255)[0]
    left_radius = center_x - row_pixels[0]
    right_radius = row_pixels[-1] - center_x

    # Top and bottom
    col_pixels = np.where(mask[:, center_x] == 255)[0]
    top_radius = center_y - col_pixels[0]
    bottom_radius = col_pixels[-1] - center_y

    return center_x, center_y, left_radius, right_radius, top_radius, bottom_radius

# Inputs
image_path = "4ab.jpg"
resize_factor = 0.25  # Resize image for performance
distance_to_globe = 5.0  # Distance to the globe in meters
focal_length = 1931.43  # Focal length in pixels

# Lower and upper bounds for color thresholding
lower_bounds = [
    np.array([10, 0, 0]),
    np.array([0, 0, 200])
]
upper_bounds = [
    np.array([150, 250, 200]),
    np.array([180, 50, 255])
]

# Read and resize the image
frame = cv2.imread(image_path)
if frame is None:
    raise ValueError("Unable to read the image.")
height, width, channels = frame.shape
frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

# Apply color thresholding and refined circle detection
binary_mask = color_thresholding(frame, lower_bounds, upper_bounds)
circle_mask = refined_circle_detection_with_mask(frame, binary_mask)

# Find center and radii
center_x, center_y, left_radius, right_radius, top_radius, bottom_radius = find_mask_center_and_radii(circle_mask)
if center_x is None:
    raise ValueError("No valid mask or radii found.")

# Convert radii to real-world meters
real_radius_left = find_real_size(focal_length, left_radius // resize_factor, distance_to_globe)
real_radius_right = find_real_size(focal_length, right_radius // resize_factor, distance_to_globe)
real_radius_top = find_real_size(focal_length, top_radius // resize_factor, distance_to_globe)
real_radius_bottom = find_real_size(focal_length, bottom_radius // resize_factor, distance_to_globe)

# Check uniformity of radii
radii = [real_radius_left, real_radius_right, real_radius_top, real_radius_bottom]
uniform = max(radii) - min(radii) < 0.01  # Uniform if difference < 1cm
ratios = [radius / max(radii) for radius in radii]

# Draw visualization
output_image = frame.copy()
#cv2.circle(output_image, (center_x, center_y), left_radius, (0, 255, 0), 2)
#cv2.circle(output_image, (center_x, center_y), 5, (0, 0, 255), -1)
cv2.line(output_image, (center_x - left_radius, center_y), (center_x + right_radius, center_y), (255, 0, 0), 2)
cv2.line(output_image, (center_x, center_y - top_radius), (center_x, center_y + bottom_radius), (255, 0, 0), 2)

# Display results
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title(
    f"Radii: Left={real_radius_left:.2f}m, Right={real_radius_right:.2f}m\n"
    f"Top={real_radius_top:.2f}m, Bottom={real_radius_bottom:.2f}m\n"
    f"Uniform: {'Yes' if uniform else 'No'}\n"
    f"Ratios: L={ratios[0]:.2f}, R={ratios[1]:.2f}, T={ratios[2]:.2f}, B={ratios[3]:.2f}"
)
plt.axis("off")
plt.show()


### c) Calculate Surface Linear Velocity
### d) Real-Time Velocity Adjustments

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Hardcoded ROI
x, y, w, h = 618, 220, 444, 386  # ROI coordinates

video_path = 'single_rotation_top_view.mp4'
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    raise ValueError("FPS could not be retrieved. Check the video file.")
frame_time = 1 / fps

# Process only the first 30 seconds
max_duration = 360  # seconds
max_frames = int(fps * max_duration)

# Rotation constants
rotation_period = 326  # 5 minutes 26 seconds (in seconds)
angular_velocity = 2 * np.pi / rotation_period  # Angular velocity (radians/second)

# Detect globe in the first frame
ret, first_frame = cap.read()
if not ret:
    raise ValueError("Unable to read video.")
gray_first = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Extract ROI from the first frame
roi_first = gray_first[y:y+h, x:x+w]

# Detect the globe using Hough Circles
circles = cv2.HoughCircles(
    roi_first,
    cv2.HOUGH_GRADIENT,
    dp=1.2,
    minDist=50,
    param1=50,
    param2=30,
    minRadius=100,
    maxRadius=250,
)

if circles is None:
    raise ValueError("No globe detected within the ROI.")
circles = np.uint16(np.around(circles))[0, 0]  # Take the first detected circle
circle_center = (circles[0], circles[1])  # (x, y) center of the circle
circle_radius = circles[2]  # Radius of the globe

# Create a circular mask for the globe
mask = np.zeros_like(roi_first, dtype=np.uint8)
cv2.circle(mask, circle_center, circle_radius, 255, -1)

# Use SIFT for feature detection within the mask (only in the first frame)
sift = cv2.SIFT_create(nfeatures=5000)
kp = sift.detect(roi_first, mask)
if len(kp) == 0:
    raise ValueError("No features detected within the globe mask.")

# Convert keypoints to numpy array of shape (N, 2), adjusting to global coords
features = np.array([kp_.pt for kp_ in kp], dtype=np.float32)
features_global = features.copy()
features_global[:, 0] += x
features_global[:, 1] += y

# Enforce maximum number of tracked features
max_features = 100  # Limit for maximum tracked features
if len(features_global) > max_features:
    features_global = features_global[np.random.choice(len(features_global), max_features, replace=False)]

# Ensure all selected features are inside the circular mask
distances = np.linalg.norm(features_global - np.array([x + circle_center[0], y + circle_center[1]]), axis=1)
features_global = features_global[distances <= circle_radius]

# Initialize tracking
all_positions = [features_global]
prev_gray = gray_first
frame_count = 0

# Precompute velocity map using the mask radius
grid_size_x, grid_size_y = 50, 50  # Grid resolution
velocity_map = np.zeros((grid_size_y, grid_size_x))  # Initialize velocity map

# Calculate tangential velocity for each grid cell
y_center = grid_size_y // 2  # Vertical center of the grid (equator)
for j in range(grid_size_y):
    for i in range(grid_size_x):
        relative_y = (j - y_center) / (grid_size_y // 2)  # Normalized latitude
        if abs(relative_y) <= 1.0:
            latitude = np.arcsin(relative_y)
            velocity_map[j, i] = angular_velocity * circle_radius * np.cos(latitude)  # Tangential velocity
        else:
            velocity_map[j, i] = 0  # Outside the circle

# Plot the velocity map
plt.figure(figsize=(8, 6))
plt.imshow(velocity_map, cmap='jet', extent=(0, grid_size_x, grid_size_y, 0), norm=Normalize())
plt.colorbar(label="Velocity (pixels/s)")
plt.title("Expected Tangential Velocity Distribution")
plt.xlabel("X (pixels)")
plt.ylabel("Y (pixels)")
plt.gca().invert_yaxis()
plt.show()

# Normalize velocity map for color mapping
norm = Normalize(vmin=velocity_map.min(), vmax=velocity_map.max())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count > max_frames:
        # Stop after 30 seconds
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Track features using optical flow
    p0 = all_positions[-1].astype(np.float32)
    # Convert p0 to ROI coords for optical flow
    p0_roi = p0.copy()
    p0_roi[:, 0] -= x
    p0_roi[:, 1] -= y

    new_features, status, _ = cv2.calcOpticalFlowPyrLK(
        prev_gray[y:y+h, x:x+w],
        gray[y:y+h, x:x+w],
        p0_roi,
        None
    )

    if new_features is None:
        # No new features tracked
        break

    valid_new_roi = new_features[status.flatten() == 1]
    valid_old_roi = p0_roi[status.flatten() == 1]

    # Convert back to global coords
    valid_new = np.zeros_like(valid_new_roi)
    valid_new[:, 0] = valid_new_roi[:, 0] + x
    valid_new[:, 1] = valid_new_roi[:, 1] + y

    # Draw the points with colors based on their velocity from the velocity map
    display_frame = frame.copy()
    for new in valid_new_roi:
        grid_x = int(new[0] / (w / grid_size_x))
        grid_y = int(new[1] / (h / grid_size_y))
        if 0 <= grid_x < grid_size_x and 0 <= grid_y < grid_size_y:
            velocity = velocity_map[grid_y, grid_x]
            color = cm.jet(norm(velocity))[:3]  # Normalize velocity to color map
            color_bgr = tuple(int(c * 255) for c in color[::-1])  # Convert to BGR
            x_new, y_new = int(new[0] + x), int(new[1] + y)
            cv2.circle(display_frame, (x_new, y_new), 3, color_bgr, -1)
            cv2.putText(display_frame, f"{velocity:.1f} px/s", (x_new + 5, y_new - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Draw detected globe
    #cv2.circle(display_frame[y:y+h, x:x+w], circle_center, circle_radius, (255, 0, 0), 2)

    # Show the video with tracked points and colors
    cv2.imshow('Video with Tracked Points', display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update for next iteration
    prev_gray = gray
    all_positions.append(valid_new)

cap.release()
cv2.destroyAllWindows()